<a href="https://colab.research.google.com/github/Leakathr1n/Foundations_of_Data_Science_Cargo2000/blob/main/Project_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Foundations of Data Science project notebook**

Team members:  

Predictive Question:

# **Packages needed to run this notebook**
We install and import all necessary libaries for this notebook in the following code cell

In [10]:
library(dplyr) # needed for basic transformation on the DS
# library(readxl) # needed to open and read xlsx-files
library(tidyr) # needed for reshaping
library(ggplot2) # needed for plotting
install.packages("GGally") # for plotting
library(GGally) # for plotting
install.packages("glmnet")
library(glmnet) #for penalized regression
install.packages("FNN") #for K nearest neighbors
library(FNN)
library(stringr) # needed for working on the airport encoding
library(purrr)  # needed for working on the airport encoding
library(readr) # needed for reading the csv

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [11]:
set.seed(12345) # setting a seed so that we get the same data set split:)

# **1. Downloading and Cleaning the Data**

In [12]:
raw_data<- read_csv('https://raw.githubusercontent.com/Leakathr1n/Foundations_of_Data_Science_Cargo2000/refs/heads/main/00%20Raw%20data/c2k_data_comma.csv')


Rows: 3943 Columns: 98
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (72): i1_dep_2_p, i1_dep_2_e, i1_dep_2_place, i1_rcf_2_p, i1_rcf_2_e, i1...
dbl (26): nr, i1_legid, i1_rcs_p, i1_rcs_e, i1_dep_1_p, i1_dep_1_e, i1_dep_1...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [13]:
# system("gdown 19zj1uN65nQVoGdFHQxtxrDVxflopCAuQ6AwR-LXQSuA")
# raw_data <- read_excel("/content/c2k_data_comma.xlsx")

In [14]:
head(raw_data)

nr,i1_legid,i1_rcs_p,i1_rcs_e,i1_dep_1_p,i1_dep_1_e,i1_dep_1_place,i1_rcf_1_p,i1_rcf_1_e,i1_rcf_1_place,⋯,o_dep_3_p,o_dep_3_e,o_dep_3_place,o_rcf_3_p,o_rcf_3_e,o_rcf_3_place,o_dlv_p,o_dlv_e,o_hops,legs
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
0,5182,199,218,210,215,609,935,736,256,⋯,?,?,?,?,?,?,780,434,1,2
1,6523,844,584,90,297,700,1935,1415,431,⋯,?,?,?,?,?,?,3870,445,1,2
2,5878,4380,4119,90,280,456,905,547,700,⋯,?,?,?,?,?,?,550,1520,1,1
3,1275,759,169,240,777,173,340,577,349,⋯,?,?,?,?,?,?,3780,159,1,1
4,8117,1597,1485,150,241,411,585,612,128,⋯,?,?,?,?,?,?,4140,4797,2,1
5,9889,181,98,240,259,815,215,223,128,⋯,?,?,?,?,?,?,3960,467,1,2


## **Cleaning the data**

We are now cleaning our data and store it in clean_data

In [17]:
# removing the unnecessary last column
clean_data <- raw_data[, !names(raw_data) %in% "...99"]

In [18]:
# converting all columns into numerical, except for "factors" or categorical variables!
# this also ensures that all our "?" are converted in NA;)

clean_data <- clean_data %>%
  mutate(across(everything(), as.character)) %>%      # temporarily ensure character so that we can replace all ?
  mutate(across(everything(), ~ na_if(., "?"))) %>%   # convert "?" → NA
  mutate(
    across(!matches("id|leg|place", ignore.case = TRUE), as.numeric),  # numeric columns
    across(matches("id|leg <place", ignore.case = TRUE), as.factor)     # factor columns
  )

In [ ]:
head(clean_data)

# **2. Getting an overview of the dataset**

In [ ]:
# Let's start with a very broad overview
# summary(clean_data)

# quite a lot of missing values; let's see if we can bring some structure to this;)
# quite a lot of outliers

In [ ]:
# str(clean_data)

# **2.1 Analysing the NA's**

We have seen that our dataset has quite a lot of missing values. While they were initially encoded as "?", when we converted them into numerical values, the question mark was converted into a proper R "NA". We will now dive deeper into the missing values and explore if there is a pattern.


For convenience, we add an overview of the dataset here. This will also help us explaining the relationship between the missing values.

![Alt text](https://raw.githubusercontent.com/Leakathr1n/Foundations_of_Data_Science_Cargo2000/main/99%20Additional%20material/Overview%20of%20dataset.png)

In [ ]:
# creating subsets per each leg & outbound
subset_i1 <- clean_data[, c("nr", grep("^i1", names(clean_data), value = TRUE))]
subset_i2 <- clean_data[, c("nr", grep("^i2", names(clean_data), value = TRUE))]
subset_i3 <- clean_data[, c("nr", grep("^i3", names(clean_data), value = TRUE))]
subset_o <- clean_data[, c("nr", grep("^o", names(clean_data), value = TRUE))]

# head(subset_i1)

In [ ]:
# setting up the data sets that I want to loop through
subset_list <- list(i1 = subset_i1, i2 = subset_i2, i3 = subset_i3, o  = subset_o)

# actually analysing the dataset
na_summary_all <- purrr::map_dfr(
  names(subset_list),
  function(name) {
    df <- subset_list[[name]]

    df %>%
      summarise(across(
        everything(),
        ~ sum(is.na(.)) / n() * 100, #calculate % NA
        .names = "NA_{.col}"
      )) %>%
      pivot_longer(
        everything(),
        names_to = "Variable",
        values_to = "Percent_NA"
      ) %>%
      mutate(
        Variable = sub("^NA_", "", Variable),
        Percent_Not_NA = 100 - Percent_NA, # calculate not NA
        Subset = name
      ) %>%
      select(Subset, Variable, Percent_NA, Percent_Not_NA)
  }
)

# print(na_summary_all, n=50)

#We can see that there is some kind of pattern; just to be entirely sure, let's double-check with code;)

In [ ]:
# Plotting a heat-map!

na_clean <- na_summary_all %>%
  mutate(
    Clean_Variable = sub("^(i1_|i2_|i3_|o_)", "", Variable)
  )

my_order <- c(
  "nr", "legid", "hops",
  "rcs_e", "rcs_p", "dlv_e", "dlv_p",
  "dep_1_e", "dep_1_p", "dep_1_place", "rcf_1_e", "rcf_1_p", "rcf_1_place",
  "dep_2_e", "dep_2_p", "dep_2_place","rcf_2_e", "rcf_2_p", "rcf_2_place",
  "dep_3_e", "dep_3_p", "dep_3_place","rcf_3_e", "rcf_3_p", "rcf_3_place"
)

na_clean <- na_clean %>% mutate(Clean_Variable = factor(Clean_Variable, levels = my_order))

options(repr.plot.width = 14, repr.plot.height = 10) # plot size;)

ggplot(na_clean, aes(y = Clean_Variable, x = Subset, fill = Percent_NA)) +
  geom_tile(color = "black") +
  geom_text(aes(label = round(Percent_NA, 1))) +
  scale_fill_gradient(low = "white", high = "cyan2", name ="% Missing") +
  theme_minimal(base_size = 14) +
  theme(
    axis.text.x = element_text(angle = 45, hjust = 1),
    # panel.grid.major = element_line(color = "black"),
    # panel.grid.minor = element_line(color = "black"),
    panel.grid = element_blank()
  ) +
  labs(
    title = "Heatmap of % of Missing Data",
    y = "Variable",
    x = "Subset",
    fill = "% NA"
  )

We can see that there is some pattern with the missing values, i.e. they are NOT missing at random.

How to interpret the heat-map now:
1. We are NOT missing any unique identifiers, as indicated by the variable "nr", legids or hops variables.
2. It also seems that we have at least 1 leg per delivery which leads to having no missing values in our first subset /incoming leg and the outgoing leg
3. We can see that per hop, we always have the same missing values. E.g. hop 1 in subset i1 contains 0% missing values for all its variables (dep_1_e, dep_1_p, ..., rcf_1_place). The same applies to other hops, and the other subsets. Similarly, there is 99.4% missing values for the 3rd hop in the first incoming leg.

Therefore, we can conclude that our NA's are **not missing at random** but that there is a structure to it. We believe that this structure relates to the factor variable hop: if it says one, the transporting leg (i1-i3) only has one delivery, and therefore missing values for hop 2 and hop3. We will verify this claim in the following section:

In [ ]:
# Create a list of subsets with their hops column name
subset_list <- list(
  i1 = list(data = subset_i1, hops_col = "i1_hops"),
  i2 = list(data = subset_i2, hops_col = "i2_hops"),
  i3 = list(data = subset_i3, hops_col = "i3_hops"),
  o  = list(data = subset_o,  hops_col = "o_hops")
)

hops_summary_all <- data.frame() # Initialise empty data frame to store results

# Loop over each subset
for(sub_name in names(subset_list)) {
  df <- subset_list[[sub_name]]$data
  col <- subset_list[[sub_name]]$hops_col

  temp <- df %>%
    count(.data[[col]]) %>%
    mutate(
      Percent = n / sum(n) * 100, #calculate per-centage
      Subset = sub_name # obtain the subset / leg name
    ) %>%
    rename(Hops = .data[[col]])

  hops_summary_all <- rbind(hops_summary_all, temp)
}

# Show result
hops_summary_all <- hops_summary_all %>% select(Subset, Hops, Percent, n)
hops_summary_all

We can see that those numbers basically confirm our suspicion that the NA's are equivalent to the number of hops as indictated by the hops column!
In order to check one by one by row, we did one last check with the following function:

In [ ]:
check_hop_consistency <- function(df, hop_var, subset_name) {

  df <- df %>% filter(!is.na(.data[[hop_var]]))  # <-- ignore rows with NA in the hops variable

  hop1_cols <- grep("_1_", names(df), value = TRUE) # Identify hop-level columns
  hop2_cols <- grep("_2_", names(df), value = TRUE)
  hop3_cols <- grep("_3_", names(df), value = TRUE)

  df %>%
    mutate(
      subset = subset_name,
      hop1_ok = if_else(.data[[hop_var]] >= 1,
                        rowSums(!is.na(across(all_of(hop1_cols)))) == length(hop1_cols),
                        rowSums(is.na(across(all_of(hop1_cols)))) == length(hop1_cols)),
      hop2_ok = if_else(.data[[hop_var]] >= 2,
                        rowSums(!is.na(across(all_of(hop2_cols)))) == length(hop2_cols),
                        rowSums(is.na(across(all_of(hop2_cols)))) == length(hop2_cols)),
      hop3_ok = if_else(.data[[hop_var]] >= 3,
                        rowSums(!is.na(across(all_of(hop3_cols)))) == length(hop3_cols),
                        rowSums(is.na(across(all_of(hop3_cols)))) == length(hop3_cols)),
      all_ok = hop1_ok & hop2_ok & hop3_ok
    ) %>%
    summarise(
      subset = subset_name,
      total_rows = n(),
      violations = sum(!all_ok),
      violation_percent = 100 * violations / total_rows
    )
}

In [ ]:
result_i1 <- check_hop_consistency(subset_i1, "i1_hops", "i1") #passing the subsets, and the variable names
result_i2 <- check_hop_consistency(subset_i2, "i2_hops", "i2")
result_i3 <- check_hop_consistency(subset_i3, "i3_hops", "i3")
result_o  <- check_hop_consistency(subset_o,  "o_hops",  "o")

bind_rows(result_i1, result_i2, result_i3, result_o)

This is great, we do not have any violations. This means that we have **no missing data at random**. We can therefore proceed to checks on *outliers* and *points with high leverage*.

### **Dealing with the NA's**

In order to perform prediction, we need to tackle the missing data.
There are different ways to deal with missing data, with the easiest one simply dropping rows that contain missing data. However, this would significantly reduce the size of our data-set and will not allow us to actually analyse the different steps in the supply chain.

Instead, we have decided to replace missing values simply with a 0. From an economic and business perspective, this makes sense, as those deliveries (since they did not happen) were not unpuctual. Therefore, we can count them as punctual!

Please note that we only replace the missing values with 0 after we tranformed the data into our actual variable of interest.

# **2.2 Look at effective time variable**
Before moving to the feature engeneering, it's important to take a look at the distribution of our variables.\
Specifically, we'll take

# **2.3. Transformation**

Startpoint A:
- target: total effective time (maximum of the leg + o)
- hops per leg
- number of legs
- airport ID (place): effective time at the airport/ delay at airport T THAT SPECIFIC STEP (target encoder); no matter in whic leg it occurs; no matter the hop
- completely drop the planned time;


---------------------------------------------------------
just keeping this for completeness now, can delete later!

Startpoint B:
- target: total delay / total effective time
- delay at leg 1, 2, 3
- hops (only outgoing)
- average delay for the outgoing process
- dummy which leg had the largest delay

## 2.3.0 Log Transformation

@Francesco will ellaborate more in 2.2

In [ ]:
# clean_data <- clean_data %>% mutate(across(ends_with("_e"),log)) # putting all _e variables in log!

## 2.3.1 Generating our target variable



Generating the target variable: effective time as the maximum time between the three legs + outbound time

In [ ]:
# Creating the subsets again including the logged variables
subset_i1 <- clean_data[, c("nr", grep("^i1", names(clean_data), value = TRUE))]
subset_i2 <- clean_data[, c("nr", grep("^i2", names(clean_data), value = TRUE))]
subset_i3 <- clean_data[, c("nr", grep("^i3", names(clean_data), value = TRUE))]
subset_o <- clean_data[, c("nr", grep("^o", names(clean_data), value = TRUE))]

subset_i1$i1_total_e <- rowSums(subset_i1[grep("_e$", names(subset_i1))],na.rm = TRUE)
subset_i2$i2_total_e <- rowSums(subset_i2[grep("_e$", names(subset_i2))],na.rm = TRUE)
subset_i3$i3_total_e <- rowSums(subset_i3[grep("_e$", names(subset_i3))],na.rm = TRUE)
subset_o$o_total_e <- rowSums(subset_o[grep("_e$", names(subset_o))], na.rm = TRUE)

In [ ]:
total_e <- Reduce(function(x, y) merge(x, y, by = "nr", all = TRUE),
                  list(subset_i1, subset_i2, subset_i3, subset_o))
total_e <- total_e %>% select(i1_total_e,i2_total_e,i3_total_e,o_total_e, nr)
head(total_e)

In [ ]:
# find the largest value among the three i columns
max_i <- apply(total_e[, c("i1_total_e", "i2_total_e", "i3_total_e")], 1, max)

# add it to o_total_e and store in new column total_e
total_e$total_e <- total_e$o_total_e + max_i

head(total_e)

In [ ]:
# Distribution of total_e
total_e %>%
  summarise(
    mean  = mean(total_e),
    sd    = sd(total_e),
    q25   = quantile(total_e, 0.25),
    med   = median(total_e),
    q75   = quantile(total_e, 0.75),
    max   = max(total_e)
  )

ggplot(total_e, aes(x = total_e)) +
  geom_histogram(bins = 30, fill = "blue", alpha = 0.7) +
  labs(x = "Total effective time (minutes)", y = "Count")

The distribution is very skewed, and we have some really big outliers. However, very big delays in supply chain are not uncommon, so it is important to preserve at least a little bit of that information (while at the same time handle their impact on the prediction).\
To do so, we will take a log transformation.

In [ ]:
# log-transform
total_e <- total_e %>%
  mutate(log_total_e = log(total_e))

ggplot(total_e, aes(x = log_total_e)) +
  geom_histogram(bins = 30, fill = "blue", alpha = 0.7) +
  labs(x = "log(Total effective time)", y = "Count")

The distribution is more bell shaped, while still preserving a bit of the skeweness and the high outliers, justifying using the log-transformation. of out target variable.

qqplot for sanity check (optional, can discuss with Alessandro if it makes sense including it)

In [ ]:
qqnorm(total_e$total_e); qqline(total_e$total_e, col = "red")
qqnorm(total_e$log_total_e); qqline(total_e$log_total_e, col = "red")

## Taking a look at the airports
Since we want to keep the information about the airports, but OHE for every airport id at every step of the transportation process would be a problem because it would create way too many variables, we want to use target encoding for the airport.\
Before doing it, it's better to do a bit of EDA on the airport variable.

In [ ]:
# take logs of the effective time first since we want to target encode for a
# log-transformed variable
log_clean_data <- clean_data %>% mutate(across(ends_with("_e"),log))
head(log_clean_data)

In [ ]:
dep1_long <- log_clean_data %>%
  select(
    nr,
    i1_dep_1_e, i1_dep_1_place,
    i2_dep_1_e, i2_dep_1_place,
    i3_dep_1_e, i3_dep_1_place,
    o_dep_1_e,  o_dep_1_place
  ) %>%
  pivot_longer(
    cols = -nr,
    names_to  = c("leg", ".value"),
    names_pattern = "(i1|i2|i3|o)_dep_1_(e|place)"
  ) %>%
  rename(
    log_eff_time = e,
    airport      = place
  ) %>%
  filter(!is.na(airport))

In [ ]:
dep1_airport_stats <- dep1_long %>%
  group_by(airport) %>%
  summarise(
    n          = n(),
    mean_log_t = mean(log_eff_time),
    sd_log_t   = sd(log_eff_time),
    .groups = "drop"
  ) %>%
  arrange(desc(n))

dep1_airport_stats

In [ ]:
dep1_by_airport_leg <- dep1_long %>%          # dep1_long from before
  group_by(airport, leg) %>%
  summarise(
    n        = n(),
    mean_log_t = mean(log_eff_time),
    .groups = "drop"
  )
dep1_by_airport_leg

In [ ]:
dep1_by_airport_leg_clean <- dep1_by_airport_leg %>%
  mutate(
    airport_num = as.numeric(airport)   # "128" and "128.0" become 128
  ) %>%
  group_by(airport_num, leg) %>%        # re‑aggregate if needed
  summarise(
    n          = sum(n),
    mean_log_t = weighted.mean(mean_log_t, n),  # or simple mean(...)
    .groups = "drop"
  ) %>%
  mutate(airport = factor(airport_num))

airport_with_multi_legs <- dep1_by_airport_leg_clean %>%
  group_by(airport) %>%
  summarise(n_legs = n_distinct(leg), .groups = "drop") %>%
  filter(n_legs >= 2)

dep1_plot_data <- dep1_by_airport_leg_clean %>%
  inner_join(airport_with_multi_legs, by = "airport")

dep1_plot_data %>%
  filter(n >= 20) %>%                        # optional: min obs per airport–leg
  ggplot(aes(x = airport, y = mean_log_t, fill = leg)) +
  geom_col(position = position_dodge(width = 0.7)) +
  labs(x = "Airport id (DEP1)", y = "Mean log time") +
  theme_minimal() +
  theme(
    axis.text.x = element_text(angle = 90, vjust = 0.5, hjust = 1, size = 6)

##2.3.2 Hops per leg & # legs

In [ ]:
hops_legs <- clean_data %>% select(nr, i1_hops, i2_hops, i3_hops, o_hops, legs)
#hops_legs

## 2.3.3 Airport ID - Train and Test Split

We want to target encode our airport IDs with their average effective time per step in the process. We have airport information given for the steps RCF and DEP for each leg.

We specifically keep the split between RCF and DEP as we believe there might be a difference how each airport performs at a different step.

Since we need to be careful about **information leakage into our test dataset**, we will already create a train and test split:

1. Train: 70
2. Test: 30

We will now impute the averages across the training data set, and then impose them on the test and training dataset.

This means that we will tune our parameters in the following steps using cross-validation, and not validation.


### Train - Test Split

In [ ]:
n <- nrow(clean_data) #count the rows in our clean data set
train_indices <- sample(1:n, size = 0.7 * n) # selected indices

train_data <- clean_data %>% slice(train_indices)
test_data <- clean_data %>% slice(-train_indices)

print(train_data)
print(test_data)

### Airport ID

In [ ]:
# we are now working off of the train data set to avoid leakage!
airports <- train_data %>%
  select(
    nr,                  # Selects the exact column 'nr'
    ends_with("_place"), # Selects columns ending in _place
    ends_with("_e")
  )

print(airports) # verify that we are working with the train data set

1. RCF

In [ ]:
rcf <- airports %>% select(grep("rcf", names(airports), value = TRUE)) # only pull rcf values

rcf_list <- list() # empty list to store the data sets
list_index <- 1 #
prefixes <- c("i1", "i2", "i3", "o") # prefixes for selecting the columns

for (prefix in prefixes) { # loop through legs
  for (hop in 1:3) { # loop through hops
    place_col <- paste0(prefix, "_rcf_", hop, "_place")
    e_col <- paste0(prefix, "_rcf_", hop, "_e")
    if (all(c(place_col, e_col) %in% names(rcf))) { # checking both columns exist

      # Select the pair, rename the columns for consistency, and filter out NAs
      temp_df <- rcf %>%
        select(Airport_ID = all_of(place_col), Effective_Time = all_of(e_col)) %>%
        filter(!is.na(Airport_ID) & !is.na(Effective_Time)) # Remove rows where either the ID or the Time is missing

      # Add the temporary dataset to our list
      rcf_list[[list_index]] <- temp_df
      list_index <- list_index + 1
    }
  }
}

df_appended_rcf_manual <- bind_rows(rcf_list) # append into one data set!

In [ ]:
# now, let's obtain the mean!
airport_summary_stats_rcf <- df_appended_rcf_manual %>%

  group_by(Airport_ID) %>%
  summarize(
    Observation_Count = n(), # Counts how many observations went into the average
    Average_Effective_Time = mean(Effective_Time, na.rm = TRUE),
    Median_Effective_Time = median(Effective_Time, na.rm = TRUE),
    .groups = 'drop'
  )

head(airport_summary_stats_rcf) # View the resulting summary table

2. DEP

In [ ]:
dep <- airports %>% select(grep("dep", names(airports), value = TRUE)) # only pull dep values

dep_list <- list() # empty list to store the data sets
list_index <- 1 #
prefixes <- c("i1", "i2", "i3", "o") # prefixes for selecting the columns

for (prefix in prefixes) { # loop through legs
  for (hop in 1:3) { # loop through hops
    place_col <- paste0(prefix, "_dep_", hop, "_place")
    e_col <- paste0(prefix, "_dep_", hop, "_e")
    if (all(c(place_col, e_col) %in% names(dep))) { # checking both columns exist

      # Select the pair, rename the columns for consistency, and filter out NAs
      temp_df <- dep %>%
        select(Airport_ID = all_of(place_col), Effective_Time = all_of(e_col)) %>%
        filter(!is.na(Airport_ID) & !is.na(Effective_Time)) # Remove rows where either the ID or the Time is missing

      # Add the temporary dataset to our list
      dep_list[[list_index]] <- temp_df
      list_index <- list_index + 1
    }
  }
}

df_appended_dep_manual <- bind_rows(dep_list) # append into one data set!

In [ ]:
# now, let's obtain the mean!
airport_summary_stats_dep <- df_appended_dep_manual %>%

  group_by(Airport_ID) %>%
  summarize(
    Observation_Count = n(), # Counts how many observations went into the average
    Average_Effective_Time = mean(Effective_Time, na.rm = TRUE),
    Median_Effective_Time = median(Effective_Time, na.rm = TRUE),
    .groups = 'drop'
  )

head(airport_summary_stats_dep) # View the resulting summary table

In [ ]:
# select our variables of interest;)
airport_summary_stats_dep <- airport_summary_stats_dep%>%select(Airport_ID, Average_Effective_Time)
airport_summary_stats_rcf <- airport_summary_stats_rcf %>%select(Airport_ID, Average_Effective_Time)

## **2.3.4 Bringing new data set together!**

In [ ]:
total_e <- total_e %>%select(total_e, nr) # select variables that we need
dataset <- merge(total_e, hops_legs, by = "nr")

head(dataset) # merge in the airports and replace the airpots with their effective time

In [ ]:
# replacing the airport IDs with the means calculated on the training data -set

airports <- clean_data %>%
  select(
    nr,                  # Selects the exact column 'nr'
    ends_with("_place") # Selects columns ending in _place
  )

airports_replaced <- airports %>%
  mutate(
    across(
      ends_with("_place"),
      ~ {
        colname <- cur_column()

        # If column contains "dep", use airport_summary_stats_dep, else use rcf
        lookup <- if (str_detect(colname, "dep")) {
          airport_summary_stats_dep
        } else {
          airport_summary_stats_rcf
        }

        # Join each column of airport IDs to get Average_Effective_Time
        lookup_match <- lookup %>%
          select(Airport_ID, Average_Effective_Time)

        # Replace ID with average time
        lookup_match$Average_Effective_Time[match(., lookup_match$Airport_ID)]
      }
    )
  )


In [ ]:
dataset <- dataset %>% left_join(airports_replaced, by = "nr")

We have decided to replace missing values simply with a 0. From an economic and business perspective, this makes sense, as those deliveries (since they did not happen) were not unpuctual. Therefore, we can count them as punctual!

In [ ]:
dataset <- dataset %>% mutate(across(everything(), ~ replace_na(., 0))) # replacing all NA's with 0

head(dataset)

In [ ]:
write.csv(dataset, file = "final_dataset.csv", row.names = FALSE)

**@TEAM:**
Here is the train and test data sets! Note that I generate the splits (and the index we use to split) in 2.3.3 Train - Test Split.

I then filter the data and only work on the train data for obtaining the average. I then impose this on the full data-set (clean_data). As we keep the same index for splitting into train & test here, there should be no leakage;)

In [ ]:
train_data <- dataset %>% slice(train_indices)
test_data <- dataset %>% slice(-train_indices)

In [ ]:
# Verifiying but should be good
#head(train_data)
#head(test_data)

#train_data %>% filter(nr == 0) # perfect;)

# **3. Descriptive Stats**
Francesco

## **3.1 Correlation**
heat map
Francesco

# **4. Predictive task**

## **4.1 Linear Regression**
Lea

## **4.2 Penalised Regressions**
Thibault

### Ridge

### Lasso

### Elastic net

## 4.3 Non-parametric Models

###KNN
Katherine

### Random Forest
Katherine

### Boosting

# Code Archive

In [ ]:
# it is a bit annoying with creating loops in google colab since you cannot easily export graphics from this notebook to the google drive :((

options(repr.plot.width=30,repr.plot.height=10) # adjusting the plot size so that is better to see in google colab


na_summary <- subset_i1 %>%
  summarise(across(everything(),
                   ~sum(is.na(.))/n() * 100,
                   .names = "NA_{.col}")) %>%
  pivot_longer(everything(),
               names_to = "Variable",
               values_to = "Percent_NA") %>%
  mutate(Variable = gsub("^NA_", "", Variable),
         Percent_Not_NA = 100 - Percent_NA) %>%
  pivot_longer(cols = c(Percent_NA, Percent_Not_NA),
               names_to = "Type",
               values_to = "Percent")

na_summary$Type <- recode(na_summary$Type,
                          Percent_NA = "Missing (NA)", #label cleaning for display
                          Percent_Not_NA = "Not Missing") # label cleaning for display

# actually plotting yeyyy;)
ggplot(na_summary, aes(x = reorder(Variable, -Percent), y = Percent, fill = Type)) +
  geom_col(position = "stack") +
  geom_text(aes(label = sprintf("%.4f%%", Percent)), #specifying how many decimals
            position = position_stack(vjust = 0.5),
            color = "black", size = 3.5, fontface = "bold") +
  labs(
    title = "Percentage of Missing vs Non-Missing Values by Column",
    x = "Variable",
    y = "Percentage (%)"
  ) +
  # scale_fill_manual(values = c("Missing (NA)" = "firebrick", "Not Missing" = "steelblue")) + #discuss colours we want!
  theme_minimal(base_size = 13) +
  theme(axis.text.x = element_text(angle = 45, hjust = 1))

 hops_summary <- subset_i1 %>%
    count(i1_hops) %>%
    mutate(Percent = n / sum(n) * 100)

    print(hops_summary)


In [ ]:
# Histograms for the variables
# Convert to long format
long_data <- subset_data %>%
  pivot_longer(
    cols = everything(),
    names_to = "Variable",
    values_to = "Value"
  )

ggplot(long_data, aes(x = Value)) +
  geom_histogram(fill = "steelblue", color = "black", bins = 30) +
  facet_wrap(~Variable, scales = "free") +
  theme_minimal()

In [ ]:
# identifying outliers based on IQR rules!

numeric_df <- subset_final %>% select(where(is.numeric))

# Empty list to store results
results <- list()

# Loop through each numeric variable
for (var in names(numeric_df)) {

  x <- numeric_df[[var]]

  Q1 <- quantile(x, 0.25, na.rm = TRUE)
  Q3 <- quantile(x, 0.75, na.rm = TRUE)
  IQR_value <- IQR(x, na.rm = TRUE)

  lower <- Q1 - 1.5 * IQR_value
  upper <- Q3 + 1.5 * IQR_value

  idx <- which(x < lower | x > upper)

  # Only add if there are outliers
  if (length(idx) > 0) {
    df_temp <- tibble(
      variable = var,
      row = idx,
      value = x[idx]
    )
    results[[length(results) + 1]] <- df_temp
  }
}

# Bind all results into one dataframe
outliers_df <- bind_rows(results)

outliers_df

**2. Outliers and Points with High Leverage**
Definition outlier:  "An outlier is a point for which $y_i$ is far from the value predicted by the model. Outliers can arise for a variety of reasons, such as incorrect recording of an observation during data collection " (James, Hastie,  Witten, Tibshirani, 2023).

We can identify plotting X against Y *or* by plotting the fitted values against the residuals. We can also identify using boxplots or histograms.

Definition *high leverage*: "Points with *high leverage* are observations with an unusual value for $x_i$"(James, Hastie,  Witten, Tibshirani, 2023). They have a sizable, and often larger impact on the fit than outliers.

In [ ]:
# select columns containing "rcs" or "dvl" - "final" columns
cols_final <- names(clean_data) %>%
  grep("(rcs|dlv)_(p|e)$", ., value = TRUE) %>% # keep only rcs or dlv columns
  setdiff(grep("_place", ., value = TRUE)) # exclude columns containing "_place"

# Subset the dataset
subset_final <- clean_data %>% select(all_of(cols_final),nr)

head(subset_final)

In [ ]:
# select columns containing "rcf or "dep" - intermediate steps
cols_inter <- names(clean_data) %>%
  grep("(dep|rcf)_\\d+_(p|e)$", ., value = TRUE) %>% # keep only dep or rcf columns
  setdiff(grep("_place", ., value = TRUE)) # exclude columns containing "_place"

# Subset the dataset
subset_inter <- clean_data %>% select(all_of(cols_inter), nr)

head(subset_inter)

In [ ]:
plot_pe_pairs <- function(subset_data) {

  options(repr.plot.width = 10, repr.plot.height = 8)
  # Identify _p and matching _e columns inside the subset
  p_cols <- grep("_p$", names(subset_data), value = TRUE)
  e_cols <- gsub("_p$", "_e", p_cols)

  # Loop over pairs
  for(i in seq_along(p_cols)) {

    df_pair <- subset_data %>%
      select(all_of(c(p_cols[i], e_cols[i]))) %>%
      rename(
        p = all_of(p_cols[i]),
        e = all_of(e_cols[i])
      )

    g <- ggplot(df_pair, aes(x = p, y = e)) +
      geom_point(color = "steelblue") +
      geom_abline(slope = 1, intercept = 0, linetype = "dashed", color = "red") +
      labs(
        title = paste0("Scatter plot: ", p_cols[i], " vs ", e_cols[i]),
        x = p_cols[i],
        y = e_cols[i]
      ) +
      theme_minimal()

    print(g)
  }
}

1. Starting with RCS and DVL

In [ ]:
plot_pe_pairs(subset_final)

Let's try and see if we can find a pattern in the outliers

Answer: not really

In [ ]:
# specific outlier analysis based on looking at the scatter plots!

outliers_final <- subset_final %>% filter(i2_dlv_e > 40000 | i3_dlv_e > 40000 | o_dlv_e > 40000)

head(outliers_final, n=20)

In [ ]:
# heatmap to see if there are any correlations between the outliers

numeric_df <- outliers_final %>%
  select(where(is.numeric))

# Compute correlation matrix
cor_matrix <- cor(numeric_df, use = "pairwise.complete.obs")

cor_long <- cor_matrix %>%
  as.data.frame() %>%
  tibble::rownames_to_column("Var1") %>%
  pivot_longer(-Var1, names_to = "Var2", values_to = "Correlation")

ggplot(cor_long, aes(x = Var1, y = Var2, fill = Correlation)) +
  geom_tile() +
  scale_fill_gradient2(low = "blue", mid = "white", high = "red",
                       midpoint = 0, limit = c(-1, 1)) +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1)) +
  labs(title = "Correlation Heatmap",
       x = "",
       y = "")

2. Continuing with DEP and RCF

In [ ]:
plot_pe_pairs(subset_inter)

Transformation: Applying the ln

In [ ]:
log_modulus <- function(x) {sign(x) * log(abs(x) + 0.1)}

In [ ]:
# Vector of leg prefixes to match
leg_prefixes <- c("i1", "i2", "i3", "o")

# Select columns matching any of the leg prefixes
cols_to_transform <- grep(paste0("^((", paste(leg_prefixes, collapse = "|"), "))_.*delay$"), names(delay_data), value = TRUE)

# Create copies for transformations
log_delay_data <- delay_data
sqrt_delay_data <- delay_data

# Apply log_modulus transform
log_delay_data[cols_to_transform] <- lapply(delay_data[cols_to_transform], function(col) log_modulus(col))

# Apply ssqrt transform
sqrt_delay_data[cols_to_transform] <- lapply(delay_data[cols_to_transform], function(col) ssqrt(col))

In [ ]:
# Group prefixes for delays
group_prefixes <- c("i1_", "i2_", "i3_", "o_")

# Loop through each group, create long format, and plot
for (prefix in group_prefixes) {
  # Select only delay columns for the current group
  group_cols <- grep(paste0("^", prefix, ".*_delay$"), names(log_delay_data), value = TRUE)

  # Pivot longer for this group
  group_long <- log_delay_data %>%
    select(all_of(group_cols)) %>%
    pivot_longer(everything(), names_to = "segment", values_to = "delay")

  # Plot histograms for all delay columns in this group
  print(
    ggplot(group_long, aes(x = delay)) +
      geom_histogram(bins = 30, fill = "blue", color = "black", alpha = 0.65) +
      facet_wrap(~segment, scales = "free") +
      theme_minimal() +
      labs(title = paste0("Delay Distributions: ", prefix, " group"),
           x = "Delay (log_minutes)", y = "Count")
  )
}

**2.2 Transformation of the data**

Startpoint A:
- target: total effective time (maximum of the leg + o)
- hops per leg
- number of legs
- airport ID: effective time at the airport/ delay at airport AT THAT SPECIFIC STEP (target encoder); no matter in whic leg it occurs; no matter the hop
- completely drop the planned time;

Startpoint B:
- target: total delay / total effective time
- delay at leg 1, 2, 3
- hops (only outgoing)
- average delay for the outgoing process
- dummy which leg had the largest delay


**2.3 Outliers and Points with High Leverage**

Definition outlier: "An outlier is a point for which  yi  is far from the value predicted by the model. Outliers can arise for a variety of reasons, such as incorrect recording of an observation during data collection " (James, Hastie, Witten, Tibshirani, 2023).

We can identify plotting X against Y or by plotting the fitted values against the residuals. We can also identify using boxplots or histograms.

Definition high leverage: "Points with high leverage are observations with an unusual value for  xi "(James, Hastie, Witten, Tibshirani, 2023). They have a sizable, and often larger impact on the fit than outliers

In [ ]:
# Summary stats

summary_stats <- delay_data %>%
  summarise(across(matches("_delay$"), # Apply to columns ending with '_delay'
                   list(mean = ~mean(., na.rm=TRUE),
                        median = ~median(., na.rm=TRUE),
                        sd = ~sd(., na.rm=TRUE)))) %>%
  pivot_longer(cols = everything(),
               names_to = c("segment", ".value"),
               names_pattern = "(.*)_(.*)$")

summary_stats

In [ ]:
# Group prefixes for delays
group_prefixes <- c("i1_", "i2_", "i3_", "o_")

# Loop through each group, create long format, and plot
for (prefix in group_prefixes) {
  # Select only delay columns for the current group
  group_cols <- grep(paste0("^", prefix, ".*_delay$"), names(delay_data), value = TRUE)

  # Pivot longer for this group
  group_long <- delay_data %>%
    select(all_of(group_cols)) %>%
    pivot_longer(everything(), names_to = "segment", values_to = "delay")

  # Plot histograms for all delay columns in this group
  print(
    ggplot(group_long, aes(x = delay)) +
      geom_histogram(bins = 30, fill = "blue", color = "black", alpha = 0.65) +
      facet_wrap(~segment, scales = "free") +
      theme_minimal() +
      labs(title = paste0("Delay Distributions: ", prefix, " group"),
           x = "Delay (minutes)", y = "Count")
  )
}

Some of the data is really skewed. To handle this, we could drop the outliers or use a transformation. \
IMO transforming the data is better option because my guess is that extreme outliers in this setting are not errors but real delays, which carry important information. \
Since we have negative values, we can't directly apply a log transformation or a sqrt, but we need to use a transformation which preserves negative values. Two options:
$$
l(x)=sign(x)*log(|x|+0.1)
$$
(the $0.1$ is added to values equal to 0, if any)
Or
$$
l(x)=sign(x)*sqrt(|x|)
$$


**LR**:
1. should we maybe apply the transformation directly to planned and effective time? Then we can avoid this, but I am open to whatever!
2. I think handling outliers should come before we apply any transformation by the way, i.e. before we create the difference variable IMO, but this is just a minor comment and structure, not the actual handling

In [ ]:
# creating our square root-function
ssqrt <- function(x) {sign(x) * sqrt(abs(x))}

In [ ]:
# Vector of leg prefixes to match
leg_prefixes <- c("i1", "i2", "i3", "o")

# Select columns matching any of the leg prefixes
cols_to_transform <- grep(paste0("^((", paste(leg_prefixes, collapse = "|"), "))_.*delay$"), names(delay_data), value = TRUE)

# Create copies for transformations
sqrt_delay_data <- delay_data

# Apply ssqrt transform
sqrt_delay_data[cols_to_transform] <- lapply(delay_data[cols_to_transform], function(col) ssqrt(col))

In [ ]:
head(sqrt_delay_data)

In [ ]:
# Group prefixes for delays
group_prefixes <- c("i1_", "i2_", "i3_", "o_")

# Loop through each group, create long format, and plot
for (prefix in group_prefixes) {
  # Select only delay columns for the current group
  group_cols <- grep(paste0("^", prefix, ".*_delay$"), names(sqrt_delay_data), value = TRUE)

  # Pivot longer for this group
  group_long <- sqrt_delay_data %>%
    select(all_of(group_cols)) %>%
    pivot_longer(everything(), names_to = "segment", values_to = "delay")

  # Plot histograms for all delay columns in this group
  print(
    ggplot(group_long, aes(x = delay)) +
      geom_histogram(bins = 30, fill = "blue", color = "black", alpha = 0.65) +
      facet_wrap(~segment, scales = "free") +
      theme_minimal() +
      labs(title = paste0("Delay Distributions: ", prefix, " group"),
           x = "Delay (sqrt_minutes)", y = "Count")
  )
}

2.1 Creating Variable of Interest: Delay of shipments

We aim to predict the delay of certain shipments. Our data set only gives us information about **planned** and **effective** duration. In order to predict the **delay**, we need to create this variable.

In [ ]:
# Get vectors of planned and effective column names (matched)
planned_cols <- grep('_p$', names(clean_data), value = TRUE)
effective_cols <- grep('_e$', names(clean_data), value = TRUE)

for (i in seq_along(planned_cols)) {
  p_col <- planned_cols[i]
  e_col <- effective_cols[i]
  delay_col <- sub('_p$', '_delay', p_col)

  # Create delay column
  clean_data[[delay_col]] <- clean_data[[e_col]] - clean_data[[p_col]]

  # Get current column order
  cols <- names(clean_data)

  # Find position of effective column
  e_pos <- which(cols == e_col)

  # Remove delay column from current position
  cols <- cols[cols != delay_col]

  # Insert delay column after effective column
  cols <- append(cols, delay_col, after = e_pos)

  # Reorder dataframe columns to this new order
  clean_data <- clean_data[, cols]
}

In [ ]:
head(clean_data) # looks correct! :)) # YEYYYY

In [ ]:
# Drop all the planned and effective columns
delay_data <- clean_data %>% select(-matches("_e$"), -matches("_p$"))

In [ ]:
head(delay_data)

In [ ]:
#Replacing NAs by 0 for all the columns ending with "delay"
delay_data_test <- delay_data %>%
  mutate(across(
    .cols = c(where(is.numeric), contains("place")), # numeric columns and those that contain a "place"
    .fns  = ~ replace(.x, is.na(.x), 0)
  ))

#Creating total_delay for each leg
delay_data_test <- delay_data_test %>%
  mutate(
    i1_total_delay = i1_rcs_delay + i1_dep_1_delay + i1_rcf_1_delay +
                     i1_dep_2_delay + i1_rcf_2_delay + i1_dep_3_delay +
                     i1_rcf_3_delay + i1_dlv_delay,

    i2_total_delay = i2_rcs_delay + i2_dep_1_delay + i2_rcf_1_delay +
                     i2_dep_2_delay + i2_rcf_2_delay + i2_dep_3_delay +
                     i2_rcf_3_delay + i2_dlv_delay,

    i3_total_delay = i3_rcs_delay + i3_dep_1_delay + i3_rcf_1_delay +
                     i3_dep_2_delay + i3_rcf_2_delay + i3_dep_3_delay +
                     i3_rcf_3_delay + i3_dlv_delay,

    o_total_delay  = o_rcs_delay + o_dep_1_delay + o_rcf_1_delay +
                     o_dep_2_delay + o_rcf_2_delay + o_dep_3_delay +
                     o_rcf_3_delay + o_dlv_delay
  )%>%
  #Reordering the new total delay variables for clarity.
  relocate(i1_total_delay, .after = i1_dlv_delay) %>%
  relocate(i2_total_delay, .after = i2_dlv_delay) %>%
  relocate(i3_total_delay, .after = i3_dlv_delay) %>%
  relocate(o_total_delay,  .after = o_dlv_delay)%>%

  #Now creating the final variable total_delay (sum of all legs)
  mutate(total_delay = i1_total_delay + i2_total_delay + i3_total_delay + o_total_delay)

delay_data_test %>%
  select(contains("total_delay"))%>%
  head()

I created the variables *_total_delay* in the dataset "delay_data_test" rather than in the original delay_data because their construction requires replacing all NA values with 0. However, the original dataset "delay_data" contains NAs that may need to be preserved for the correlations analyses below. To avoid overwriting these missing values, I temporarily duplicated the data into "delay_data_test", applied the NA → 0 transformation only there, and computed the total delay variables in this safe copy. This test dataset will be used for the prediction models (Ridge, LASSO, Elastic Net) while we decide as a group when to process missing values in the final workflow.

--------------------------------------------

replace the NA's with 0!

**2.4 Analysing the correlation between the variables**

2.4.1 Numerical variables

In [ ]:
# Calculate correlations
delay_cols <- grep("_delay$", names(sqrt_delay_data), value = TRUE)
delay_corr_data <- sqrt_delay_data[delay_cols]
corr_matrix <- cor(delay_corr_data, use = "pairwise.complete.obs")

# Convert the correlation matrix to long format for ggplot
corr_df <- as.data.frame(corr_matrix)
corr_df$Var1 <- rownames(corr_df)
corr_long <- corr_df %>%
  pivot_longer(-Var1, names_to = "Var2", values_to = "correlation")

# Plot the heatmap
ggplot(corr_long, aes(x = Var1, y = Var2, fill = correlation)) +
  geom_tile() +
  scale_fill_gradient2(low = "blue", high = "red", mid = "white",
                       midpoint = 0, limit = c(-1, 1), space = "Lab",
                       name="Correlation") +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1)) +
  labs(title = "Delay Column Correlation Heatmap", x = "", y = "")

**TO-DO**

1. grey means too little observations
2. negative correlation: if there is one delay, the following step will try to recover this
3. positive correlation: positive correlation across different legs
4. syncronisation of legs?

2.4.2 By airports

**3. Prediction**


Preparing data for prediction
In order to run prediction algorithms, we need to tackle missing data. As ellaborated in *section 2.1 Analysing the NA's*, we will replace all missing values with a 0.

In [ ]:
sqrt_delay_data <- sqrt_delay_data %>%
  mutate(across(
    .cols = c(where(is.numeric), contains("place")), # numeric columns and those that contain a "place"
    .fns  = ~ replace(.x, is.na(.x), 0)
  ))

In [ ]:
sqrt_delay_data

In [ ]:
# Team: please use the following data set:
summary(sqrt_delay_data)

I temporarily apply the square-root transformation to "delay_data_test" (instead of the original "delay_data") because this is the dataset where missing values were replaced with zeros and where the total_delay_* variables were created. Keeping all processing steps in the same dataset avoids mixing raw and transformed values and ensures that all delay variables—both individual delays and aggregated delays—are on the same scale before running Ridge, LASSO, and Elastic Net.

This setup is temporary: after discussing with the group, we will adjust the code to apply the transformation directly on the final version of delay_data.

In [ ]:
# creating our square root-function
ssqrt <- function(x) {sign(x) * sqrt(abs(x))}

# Vector of leg prefixes to match
leg_prefixes <- c("i1", "i2", "i3", "o")

# Select columns
cols_to_transform <- c(
  grep(paste0("^((", paste(leg_prefixes, collapse = "|"), "))_.*delay$"), names(delay_data_test), value = TRUE), # Select columns matching any of the leg prefixes
  grep("^total_delay", names(delay_data_test), value = TRUE) #Select the columns capturing the total_delay
)

# Create copies for transformations
sqrt_delay_data_test <- delay_data_test

# Apply ssqrt transform
sqrt_delay_data_test[cols_to_transform] <- lapply(delay_data_test[cols_to_transform], function(col) ssqrt(col))

sqrt_delay_data_test

In [ ]:
summary(sqrt_delay_data_test)

3.1 Classic Linear Regression

3.2 Linear Models

3.2.1 Setting Up the Data

In [ ]:
#Define the response variable and features for our predictive model

# Response variable (y): the total delay of each shipment
y <- sqrt_delay_data_test$total_delay

# Feature variables (X): all step-by-step delays and aggregated delays per leg
features <- setdiff(grep("_delay$", names(sqrt_delay_data_test), value = TRUE), "total_delay")

#Subset the dataset to only keep the feature columns
X <- sqrt_delay_data_test[, features]

In [ ]:
#Quick check
dim(X)
length(y)
#Looks alright

In [ ]:
# Splitting into train data and test data (70/30)

set.seed(2025)  # ensure we get the same split every time

# Randomly sample 70% of the row indices for the training set
n <- nrow(sqrt_delay_data_test)
train_indices <- sample(1:n, size = 0.7 * n)

# Create training and test sets
X_train <- X[train_indices, ]   # features for training
X_test  <- X[-train_indices, ]  # features for testing

y_train <- y[train_indices]     # response for training
y_test  <- y[-train_indices]    # response for testing

In [ ]:
#Quick check
dim(X_train)
length(y_train)
dim(X_test)
length(y_test)
#Looks alright as well

A standardization of the features is necessary in penalized regressions like Ridge, LASSO, or Elastic Net to ensure that the penalty applies fairly across all variables, regardless of their original scale.

In our dataset, this is particularly important for two reasons:
1) some delay variables have very negative values while others are more centered around zero, creating large differences in scale;
2) some features, such as i1_total_delay, i2_total_delay, i3_total_delay, and o_total_delay, are sums of other delay variables, giving them naturally larger magnitudes.

Standardizing all features puts them on a comparable scale, preventing variables with larger ranges from dominating the penalty and ensuring meaningful coefficient estimates.

In [ ]:
# Compute mean and standard deviation of each feature on the training set
feature_means <- apply(X_train, 2, mean)
feature_sds   <- apply(X_train, 2, sd)

# Standardize training set
X_train_scaled <- scale(X_train, center = feature_means, scale = feature_sds)

# Standardize test set using the training set statistics
X_test_scaled <- scale(X_test, center = feature_means, scale = feature_sds)

3.2.2 Ridge

In [ ]:
#Using Ridge Model on the training data
ridge_model <- glmnet(
  x = X_train_scaled,
  y = y_train,
  alpha = 0 #alpha = 0 for the L2 penalty (sum of squares)
)

print(ridge_model)

In [ ]:
plot(ridge_model)
abline(h=0)
grid()

In [ ]:
#Finding the best lambda
cv_ridge <- cv.glmnet(
  x = X_train_scaled,
  y = y_train,
  alpha = 0
)

print(cv_ridge)

In [ ]:
plot(cv_ridge)

We obtain two relevant values of λ for the Ridge model.

The optimal $\lambda_{min}$ (lambda.min), which minimizes the mean squared error (MSE), is 5.44. This value provides the best predictive performance for $y$. At this relatively small $\lambda$, the penalty imposed by Ridge—proportional to the sum of squared coefficients is weak, allowing the coefficients to remain relatively large. As a result, each feature retains an influence close to that of a classical OLS regression, with only slight shrinkage to prevent excessively extreme coefficients. \
While the model predicts well, it is somewhat more sensitive to small variations in the data because the coefficients are less constrained. Specifically, if some observations are atypical or contain errors, the small penalty allows the coefficients to fluctuate more in order to adapt to these points. Additionally, if features are highly correlated, some coefficients may take large values, and with a small $\lambda$, Ridge does not shrink them sufficiently, leading to potential oscillations and reduced stability on new data.


In contrast, the $\lambda_{1se}$ value corresponding to one standard error above the optimal $\lambda_{min}$ (lambda.1se) is 21.96. At this larger $\lambda_{1se}$, the penalty is stronger, and the coefficients are more heavily shrunk towards zero. This reduces the influence of highly correlated or noisy variables and makes the model less sensitive to random fluctuations in the dataset. Consequently, if cross-validation were repeated on a different sample, the coefficients would be less likely to vary substantially, improving stability and robustness.


For our case, the choice between these two $\lambda$ values depends on the characteristics of our dataset. If we found strong correlations among the features or a significant number of atypical observations, the more regularized $\lambda_{1se}$ (21.96) should be preferred. If correlations are moderate and the dataset does not contain extreme observations, using the $\lambda_{min}$ that minimizes the MSE (5.44) is reasonable, as it provides better predictive accuracy.

In [ ]:
#Extract the lambda.1se from cross-validation
best_lambda_1se <- cv_ridge$lambda.1se
best_lambda_1se
# This should return 21.96

# Predict on the test set using lambda.1se
ridge_pred_1se <- predict(cv_ridge,
                          s = best_lambda_1se,
                          newx = X_test_scaled)

#Compute the Mean Squared Error (MSE) on the test set
mse_test_1se <- mean((y_test - ridge_pred_1se)^2)
mse_test_1se


In [ ]:
#Extract the lambda.min from cross-validation
best_lambda_min <- cv_ridge$lambda.min
best_lambda_min
# This should return 5.44

# Predict on the test set using lambda.min
ridge_pred_min <- predict(cv_ridge,
                          s = best_lambda_min,
                          newx = X_test_scaled)

#Compute the Mean Squared Error (MSE) on the test set
mse_test_min <- mean((y_test - ridge_pred_min)^2)
mse_test_min


In [ ]:
summary(y_test)

The comparison with the MSE obtained at $\lambda_{min}$ (≈ 835) shows that the performance gap between the two models remains very small, on the order of 4–5%. In other words, using $\lambda_{1se}$ provides a more regularized model—hence simpler and more stable with respect to data variations—while sacrificing only a very marginal amount of predictive accuracy. This suggests that the model tuned with $\lambda_{1se}$ generalizes at least as well as, and potentially better than, the one based on $\lambda_{min}$, despite the slight increase in test-set MSE.

Importantly, these error levels remain coherent given the scale of the dependent variable: the test-set values of ($y$) span a wide range (from –167 to 745). A MSE around 800–900 corresponds to a Root Mean Squared Error of roughly 29 units, which is small relative to the total variation of the outcome variable. In this context, such error magnitudes are reasonable and indicate that the ridge model succeeds in capturing substantial structure in the data despite its inherent noise and dispersion.


=> Need to check with the MSE of the OLS Regression part 3.1


In [ ]:
# Convert predictions to numeric
ridge_pred_min_num <- as.numeric(predict(cv_ridge, s = cv_ridge$lambda.min, newx = X_test_scaled))
ridge_pred_1se_num <- as.numeric(ridge_pred_1se)

# Create a dataframe for plotting
plot_data <- data.frame(
  Observed = y_test,
  Predicted_min = ridge_pred_min_num,
  Predicted_1se = ridge_pred_1se_num
)

# Set up plotting area: 1 row, 2 columns
par(mfrow = c(1,2))

# Plot for lambda.min
plot(plot_data$Observed, plot_data$Predicted_min,
     xlab = "Observed Values (Test Set)",
     ylab = "Predicted Values (Ridge λ.min)",
     main = "Observed vs Predicted – λ.min",
     pch = 19, col = "blue")
abline(a = 0, b = 1, col = "red", lwd = 2)

# Plot for lambda.1se
plot(plot_data$Observed, plot_data$Predicted_1se,
     xlab = "Observed Values (Test Set)",
     ylab = "Predicted Values (Ridge λ.1se)",
     main = "Observed vs Predicted – λ.1se",
     pch = 19, col = "green")
abline(a = 0, b = 1, col = "red", lwd = 2)

# Reset plotting layout to default
par(mfrow = c(1,1))

In [ ]:
#Using Ridge Model on the training data
ridge_model <- glmnet(
  x = X_train,
  y = y_train,
  alpha = 0 #alpha = 0 for the L2 penalty (sum of squares)
)

print(ridge_model)

In [ ]:
plot(ridge_model)
abline(h=0)
grid()

In [ ]:
#Finding the best lambda
cv_ridge <- cv.glmnet(
  x = as.matrix(X_train),
  y = y_train,
  alpha = 0
)

print(cv_ridge)

In [ ]:
plot(cv_ridge)

We obtain two relevant values of λ for the Ridge model.

The optimal $\lambda_{min}$ (lambda.min), which minimizes the mean squared error (MSE), is 5.44. This value provides the best predictive performance for $y$. At this relatively small $\lambda$, the penalty imposed by Ridge—proportional to the sum of squared coefficients is weak, allowing the coefficients to remain relatively large. As a result, each feature retains an influence close to that of a classical OLS regression, with only slight shrinkage to prevent excessively extreme coefficients. \
While the model predicts well, it is somewhat more sensitive to small variations in the data because the coefficients are less constrained. Specifically, if some observations are atypical or contain errors, the small penalty allows the coefficients to fluctuate more in order to adapt to these points. Additionally, if features are highly correlated, some coefficients may take large values, and with a small $\lambda$, Ridge does not shrink them sufficiently, leading to potential oscillations and reduced stability on new data.


In contrast, the $\lambda_{1se}$ value corresponding to one standard error above the optimal $\lambda_{min}$ (lambda.1se) is 21.96. At this larger $\lambda_{1se}$, the penalty is stronger, and the coefficients are more heavily shrunk towards zero. This reduces the influence of highly correlated or noisy variables and makes the model less sensitive to random fluctuations in the dataset. Consequently, if cross-validation were repeated on a different sample, the coefficients would be less likely to vary substantially, improving stability and robustness.


For our case, the choice between these two $\lambda$ values depends on the characteristics of our dataset. If we found strong correlations among the features or a significant number of atypical observations, the more regularized $\lambda_{1se}$ (21.96) should be preferred. If correlations are moderate and the dataset does not contain extreme observations, using the $\lambda_{min}$ that minimizes the MSE (5.44) is reasonable, as it provides better predictive accuracy.

In [ ]:
X_test_s <- X_test

In [ ]:
X_test_s[ , grepl("o", names(X_test_s)) ] <- 0
X_test_s

In [ ]:
#Extract the lambda.1se from cross-validation
best_lambda_1se <- cv_ridge$lambda.1se
best_lambda_1se
# This should return 21.96

# Predict on the test set using lambda.1se
ridge_pred_1se <- predict(cv_ridge,
                          s = best_lambda_1se,
                          newx = as.matrix(X_test_s))

#Compute the Mean Squared Error (MSE) on the test set
mse_test_1se <- mean((y_test - ridge_pred_1se)^2)
mse_test_1se


In [ ]:
ridge_pred_1se

In [ ]:
y_test

In [ ]:
#Extract the lambda.min from cross-validation
best_lambda_min <- cv_ridge$lambda.min
best_lambda_min
# This should return 5.44

# Predict on the test set using lambda.min
ridge_pred_min <- predict(cv_ridge,
                          s = best_lambda_min,
                          newx = as.matrix(X_test))

#Compute the Mean Squared Error (MSE) on the test set
mse_test_min <- mean((y_test - ridge_pred_min)^2)
mse_test_min


In [ ]:
summary(y_test)

The comparison with the MSE obtained at $\lambda_{min}$ (≈ 835) shows that the performance gap between the two models remains very small, on the order of 4–5%. In other words, using $\lambda_{1se}$ provides a more regularized model—hence simpler and more stable with respect to data variations—while sacrificing only a very marginal amount of predictive accuracy. This suggests that the model tuned with $\lambda_{1se}$ generalizes at least as well as, and potentially better than, the one based on $\lambda_{min}$, despite the slight increase in test-set MSE.

Importantly, these error levels remain coherent given the scale of the dependent variable: the test-set values of ($y$) span a wide range (from –167 to 745). A MSE around 800–900 corresponds to a Root Mean Squared Error of roughly 29 units, which is small relative to the total variation of the outcome variable. In this context, such error magnitudes are reasonable and indicate that the ridge model succeeds in capturing substantial structure in the data despite its inherent noise and dispersion.


=> Need to check with the MSE of the OLS Regression part 3.1


In [ ]:
# Convert predictions to numeric
ridge_pred_min_num <- as.numeric(predict(cv_ridge, s = cv_ridge$lambda.min, newx = X_test))
ridge_pred_1se_num <- as.numeric(ridge_pred_1se)

# Create a dataframe for plotting
plot_data <- data.frame(
  Observed = y_test,
  Predicted_min = ridge_pred_min_num,
  Predicted_1se = ridge_pred_1se_num
)

# Set up plotting area: 1 row, 2 columns
par(mfrow = c(1,2))

# Plot for lambda.min
plot(plot_data$Observed, plot_data$Predicted_min,
     xlab = "Observed Values (Test Set)",
     ylab = "Predicted Values (Ridge λ.min)",
     main = "Observed vs Predicted – λ.min",
     pch = 19, col = "blue")
abline(a = 0, b = 1, col = "red", lwd = 2)

# Plot for lambda.1se
plot(plot_data$Observed, plot_data$Predicted_1se,
     xlab = "Observed Values (Test Set)",
     ylab = "Predicted Values (Ridge λ.1se)",
     main = "Observed vs Predicted – λ.1se",
     pch = 19, col = "green")
abline(a = 0, b = 1, col = "red", lwd = 2)

# Reset plotting layout to default
par(mfrow = c(1,1))

3.2.3 Lasso

3.2.4 Elastic Net

3.3 Non-linear Models

3.3.1 K-nearest neighbours


In [ ]:
# partitioning scaled train data from above into train and validation data
v <- nrow(X_train_scaled)
val_indices <- sample(1:v, size = 0.3 * v) # taking 30% of our scaled test data to use for validation
# this is just a choice I'm making for now so that I can use the same test split
# as the Ridge model to compare model performance on the same test split
# We should discuss this though!

# creating train and validation sets

X_train_scaled_knn <- X_train_scaled[-val_indices,] # features for training
X_val_scaled_knn <- X_train_scaled[val_indices,] # features for validation

# response variables
y_train_knn <- y_train[-val_indices] # response for training
y_val_knn <- y_train[val_indices] # response for validation




In [ ]:
# Quick Check
dim(X_train_scaled_knn)
length(y_train_knn)
dim(X_val_scaled_knn)
length(y_val_knn)

In [ ]:
# KNN set up
K <- 50 # setting a max K of 50, discuss if we want to go higher/lower

# Creating empty (for now) vectors to store MSE for each K
train_MSE <- rep(NA, K) # will hold MSE for each K on train data
val_MSE <- rep(NA, K) # will hold MSE for each K on val data

In [ ]:
# KNN train and validation loop
for (k in 1:K){ # iterates from 1 to max value of K
  # kNN with training data
  kNN_train <- knn.reg(train = X_train_scaled_knn,
                 test = X_train_scaled_knn,
                 y = y_train_knn,
                 k = k)
  # calculating train MSE for each K and storing each K's MSE in vector
  # train_MSE[k] <- mean(( - y_train_knn)^2)

  # kNN with validation data
  kNN_val  <- knn.reg(train = X_train_scaled_knn,
                test = X_val_scaled_knn,
                y = y_train_knn,
                k = k)
  # calculating validation MSE for each K and storing each K's MSE in vector
  # val_MSE[k]  <- mean((knn_val-y_val_knn)^2)
}

In [ ]:
# Evaluating values of K
# comparison between training performance and validation performance
plot(1:K,val_MSE, t ='l',ylim=c(min(val_MSE),1),col='darkblue',lwd=2)
lines(1:K,train_MSE, col='firebrick',lwd=2)
grid()
legend('top',c('Val. MSE','Train MSE'), col=c('darkblue','firebrick'),lty=1,lwd=3)
# numeric calculation of best K value
k_star <- (1:K)[which.min(val_MSE)]
# printing best model's MSE and K value
sprintf('Best model minimized MSE to %.4f with %.s neighbors', min(val_MSE), k_star)

In [ ]:
# KNN on test data using best K value from training/validation
kNN_test <- knn.reg(train = X_train_scaled_knn,
                    test = X_test_scaled,
                    y = X_train_scaled_knn,
                    k = k_star)

3.3.2 Random forest